In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
from astropy.table import Table
import astropy.units as u
from astropy import constants as const
mpl.rcParams['text.usetex'] = True

In [3]:
mu    = 1.0/(2.0*0.7 + (3.0/4.0)*0.28 + 0.02/2. )
mu_e  = 2.0/(1 + 0.7)
Rho_c = 1e6*mu_e
G = const.G.cgs.value#6.6743e-8
R = const.R.cgs.value#8.31447e7
C = 0.48#0.3639385751673097
M = 2*const.M_sun.cgs.value#2.0*1.9884e33 
x = np.linspace(1e-10,1e10,10000)
y = np.linspace(1e-10, Rho_c, 10000)
z = np.linspace(Rho_c, 1e10, 10000)
xT = np.linspace(1e5,1e9,1000)
def Rad_IdealGas(X):
    return 3.2e7*X**(1./3.)*mu**(-1./3.)

def NR_IdealGas(Y):
    return 1.21e5*mu*mu_e**(-5./3.)*Y**(2./3.)

def ER_IdealGas(Z):
    return 1.21e7*mu*mu_e**(-4./3.)*Z**(1./3.)

Sun = [150, 15.7e6]

def Evolution_Track(rho, c, g, r,mass):
    return (c*g/r)*mu*mass**(2./3.)*rho**(1./3.)

def Evolution_Track2(Tem, c, g, r,mass):
    return ((Tem*r)/(c*g*mu*(M**(2./3.))))**(3.)

## Evolution of the core

In [4]:
PP=[]
PH=[]
for i in range(1,1179):
    A=Table.read('/disks/strw3/jvilla/MESA_Juan_Jorge/2M_prems_to_wd/LOGS/profile'+str(i)+'.data',format='ascii',data_start=5,fast_reader=True)
    B=Table.read('/disks/strw3/jvilla/MESA_Juan_Jorge/2M_prems_to_wd/LOGS/profile'+str(i)+'.data',format='ascii',data_start=1,data_end=3,fast_reader=True)
    PP.append(A)
    PH.append(B)

In [5]:
# savetxt('Columns.txt',PP)
# savetxt('Rows.txt',PH)
# # PP=loadtxt('Columns.txt')
# # PH=loadtxt('Rows.txt')

In [20]:
figure(figsize=(16,12))

cmap = cm.get_cmap('rainbow', 40)

for i in range(1,len(PP)):
    if PH[i]['col5'][1] < 1.1e4:
        pa=scatter(PP[i]['col2'][-1],PP[i]['col3'][-1], s = 38, edgecolors='none', c=(float(PH[i]['col5'][1])),cmap=cmap, 
                   norm=matplotlib.colors.PowerNorm(gamma = 2., vmin=1e0, vmax=1e4))
        
    elif PH[i]['col5'][1] > 1.1e5 and PH[i]['col5'][1] < 8e5: 
        pa=scatter(PP[i]['col2'][-1],PP[i]['col3'][-1], s = 38, edgecolors='none', c=(float(PH[i]['col5'][1])),cmap=cmap, 
                   norm=matplotlib.colors.PowerNorm(gamma = 2., vmin=1.1e4, vmax=8e5))
        
    elif PH[i]['col5'][1] > 8e5: 
        pa=scatter(PP[i]['col2'][-1],PP[i]['col3'][-1], s = 38, edgecolors='none', c=(float(PH[i]['col5'][1])),cmap=cmap, 
                   norm=matplotlib.colors.PowerNorm(gamma = 2., vmin=8e5, vmax=1.2e9))        
    
tick_params('both', length = 10, width = 2, which = 'major')
rc('font', family='serif', weight = 'bold', size=20)    
cba = plt.colorbar(pa,orientation="vertical",  pad = 0.0008, aspect=30, fraction=0.05 )
cba.set_label('Age [Gyr]')
plt.plot(np.log10(xT),np.log10(Evolution_Track2(xT,C,G,R,M)),'-',color = 'k',label=r'2$\textrm{M}_\odot$ Model', linewidth = 0.7)   
plt.hlines(np.log10(Rho_c),-2,8.8, linestyle='--',color = 'green', linewidth = 2)
plt.plot(np.log10(Rad_IdealGas(x)),np.log10(x), '--',color = 'blue', linewidth = 2)
plt.plot(np.log10(NR_IdealGas(y)), np.log10(y),'--',color = 'orange', linewidth = 2)
plt.plot(np.log10(ER_IdealGas(z)),np.log10(z), '--',color = 'black', linewidth = 2)   
plot(np.log10(Sun[1]),np.log10(Sun[0]), 'o', c = 'orange',ms=8, label='Solar Core')
plt.xlabel('$\log(T_c)$', fontsize = 20, fontweight = 'bold')
plt.ylabel(r'$\log(\rho_c)$', fontsize = 20, fontweight = 'bold')
plt.xlim(5.3,9)
plt.ylim(-4,7.4)
#annotate('H ignition', xy=(7.15, 1.7), xytext=(6.6, 1.74),
  #       arrowprops=dict(arrowstyle="fancy",color="0.5",connectionstyle="arc3,rad=-0.2"))
#annotate('He ignition', xy=(7.9, 5.8), xytext=(7.06, 5.5),
 #        arrowprops=dict(arrowstyle="fancy",color="0.5",connectionstyle="arc3,rad=-0.2"))
#text(6.33, -1.72,'Pre-main sequence',  ha="center", va="center", rotation=45)
#text(7.41, 1.71,'Main sequence',  ha="center", va="center", rotation=45)
#text(7.54, 4.88,'Red Giant Branch',  ha="center", va="center", rotation=30)
#text(8.04, 4.19,'Helium flash',  ha="center", va="center")
#text(7.84, 6.61,'<-- To white dwarf',  ha="center", va="center")
legend(frameon = False, loc = 'lower right')
# title(r'Evolution of the core in the logT$_c$ - log $\rho_c$ plane')
savefig('1.png',dpi=300,bbox_inches='tight')

## HR diagram

In [19]:
figure(figsize=(16,12))

cmap = cm.get_cmap('rainbow', 40)

'''for i in range(47,len(PP)):
    pa=scatter(log10(float(PH[i]['col7'][-1])),log10(float(PH[i]['col8'][-1])),
               c=(float(PH[i]['col5'][1])),cmap='coolwarm',
               norm=matplotlib.colors.PowerNorm(gamma=4.,vmin=1e8, vmax=1.2e9))
for i in range(1,47):
    pb=scatter(log10(float(PH[i]['col7'][-1])),log10(float(PH[i]['col8'][-1])),
               c=(float(PH[i]['col5'][1])),cmap='YlGnBu',
               norm=matplotlib.colors.PowerNorm(gamma=1./4.,vmin=0, vmax=1e8))'''

for i in range(1,len(PP)):
    if PH[i]['col5'][1] < 1.1e4:
        pa=scatter(log10(float(PH[i]['col7'][-1])),log10(float(PH[i]['col8'][-1])), s = 38, edgecolors='none', c=(float(PH[i]['col5'][1])),cmap=cmap, 
                   norm=matplotlib.colors.PowerNorm(gamma = 2., vmin=1e0, vmax=1e4))
        
    elif PH[i]['col5'][1] > 1.1e5 and PH[i]['col5'][1] < 8e5: 
        pa=scatter(log10(float(PH[i]['col7'][-1])),log10(float(PH[i]['col8'][-1])), s = 38, edgecolors='none', c=(float(PH[i]['col5'][1])),cmap=cmap, 
                   norm=matplotlib.colors.PowerNorm(gamma = 2., vmin=1.1e4, vmax=8e5))
        
    elif PH[i]['col5'][1] > 8e5: 
        pa=scatter(log10(float(PH[i]['col7'][-1])),log10(float(PH[i]['col8'][-1])), s = 38, edgecolors='none', c=(float(PH[i]['col5'][1])),cmap=cmap, 
                   norm=matplotlib.colors.PowerNorm(gamma = 2., vmin=8e5, vmax=1.2e9))
        
    
tick_params('both', length = 10, width = 2, which = 'major')
rc('font', family='serif', weight = 'bold', size=20)    
cba = plt.colorbar(pa,orientation="vertical",  pad = 0.0008, aspect=30, fraction=0.05 )  
cba.set_label('Age [Gyr]')
xlim(5.18,3.4)
ylim(-0.97, 4.1)
ylabel(r'$\log(L/L_\odot)$', fontsize = 20, fontweight = 'bold')
xlabel(r'$\log(T_{eff})$', fontsize = 20, fontweight = 'bold')
# title(r'Hertzsprung-Russel diagram')
#text(3.63, 1.94,'Pre-main sequence',  ha="center", va="center", rotation=50)
#text(3.96, 1.08,'Main sequence',  ha="center", va="center", rotation=-30)
#text(3.65, 2.44,'Red Giant Branch',  ha="center", va="center", rotation=50)
# annotate('Deuterium burning', xy=(3.72, 0.62), xytext=(3.72, 0.62),arrowprops=dict(facecolor='black', shrink=0.001))
#text(3.45, 3.35,'Helium flash',  ha="center", va="center")
#text(3.81, 1.57,'Hydrogen-shell burning phase',  ha="center", va="center")
#text(3.68, 3.7,'<-- To white dwarf',  ha="center", va="center")
savefig('2.png',dpi=300,bbox_inches='tight')

## Convection pre-main

In [18]:
file=28
figure(figsize=(16,12))
plot(PP[file]['col24'],log10(PP[file]['col32']),label=r'$\Delta_{ad}$', c = 'orange', linewidth = 2)
plot(PP[file]['col24'],log10(PP[file]['col61']),label=r'$\Delta_{rad}$', c = 'blue', linewidth = 2)
fill_between(PP[file]['col24'], min(min(log10(PP[file]['col32'])),min(log10(PP[file]['col61'])))-.1,
             max(max(log10(PP[file]['col32'])),max(log10(PP[file]['col61'])))+.1,
             where=log10(PP[file]['col61']) > log10(PP[file]['col32']),
             color='green',alpha=0.18,label='Convective Zone')
xlim(-0.2, 12.2)
ylim(-1.24, 6)
ylabel(r'$\Delta$')
xlabel(r'$R/R_\odot$')
#legend(frameon = True, loc = (11.5, 4.5))
legend(frameon = True, loc = 'upper left')
# title('Convection in the pre-main sequence phase')
savefig('3.png',dpi=300,bbox_inches='tight')

## Convection main sequence

In [43]:
file=49
figure(figsize=(16,12))
plot(PP[file]['col24'],log10(PP[file]['col32']),label=r'$\Delta_{ad}$', c = 'orange', linewidth = 2)
plot(PP[file]['col24'],log10(PP[file]['col61']),label=r'$\Delta_{rad}$', c = 'blue', linewidth = 2)
fill_between(PP[file]['col24'], min(min(log10(PP[file]['col32'])),min(log10(PP[file]['col61'])))-.1,
             max(max(log10(PP[file]['col32'])),max(log10(PP[file]['col61'])))+.1,
             where=log10(PP[file]['col61']) > log10(PP[file]['col32']),
             color='green',alpha=0.18,label='Convective Zone')
xlim(0,2.45)
ylim(-1.23, 0.3)
ylabel(r'$\Delta$')
xlabel(r'$R/R_\odot$')
legend(frameon = True, loc = 'upper center')
# title('Convection in the main sequence phase')
savefig('4.png',dpi=300,bbox_inches='tight')